# Significant impact rule

In [ ]:
def opus_ir_pro(k,val,tid,num_col,measurement,adjust=0,pro=0,independent_pro=0):
    k = 100
    me_1 = np.mean(val)
    rule = [(me_1,'empty') for i in range(k)]
    heapify(rule)
    current = set()
    pro_rule = pro_a(k,me_1,rule,val,tid,num_col,current,tid.keys(),measurement,pro,adjust)
    if independent_pro:
        r = [i for i in pro_rule if len(i)>2]
        d = set()
        b = []
        for i,j in enumerate(r):
            for p,q in enumerate(r[:i]):
                s1 = set(r[i][1:-1])
                s2 = set(r[p][1:-1])
                if s2 > s1:
                    temp = [k for k in r[i][-1] if k not in r[p][-1]]
                    if me_1 >= np.mean(temp):
                        d.add(i)
                        b.append(q)
                    else:
                        t , p_val = ttest_ind(val, temp, equal_var=True)
                        if p_val > 0.05:
                            d.add(i)
                            b.append(q)            
                if s1 > s2:
                    temp = [k for k in r[p][-1] if k not in r[i][-1]]
                    if me_1 >= np.mean(temp):
                        d.add(p)
                        b.append(j)
                    else:
                        t , p_val = ttest_ind(val, temp, equal_var=True)
                        if p_val > 0.05:
                            d.add(p)
                            b.append(j)  
        d = list(d)
        d.sort()
        f = []
        for i in d[::-1]:
            f.append(r.pop(i))
        r_test = [i[:-1] for i in pro_rule if len(i)>2], [i[:-1] for i in r],b
        return r_test
    return [i[:-1] for i in pro_rule if len(i)>2]
def pro_a(k,me_1,rule,val,tid,num_col,current,available,measurement,pro=0,adjust=0):
    global counter
    sofar = set()
    for p in available:
        new  = current.copy()
        new.add(p)
        if current == set():
            upper_bound = (max([val[i] for i in tid[p]]))
            val1 = [val[i] for i in tid[p]]
            len_tid = 1
            if measurement == 'mean':
                ir = (np.mean(val1))
            if measurement == 'impact':
                ir = (np.mean(val1) - me_1) * len(tid[p]) 
        else:        
            inter = set.intersection(*map(set,[tid[i] for i in new]))
            upper_bound = (max([val[i] for i in inter]+[0]))
            len_tid = len(new)
            val1 = [val[i] for i in inter]
            if measurement == 'mean':
                ir = (np.mean(val1))
            if measurement == 'impact':
                ir = (np.mean(val1) - me_1) * len(val1) 
        min_heap = heappop(rule)
        if pro:
            if upper_bound > min_heap[0] and not math.isnan(upper_bound):
                if ir > min_heap[0]:
                    parent = [i for i in rule if i[1:-1] and set(i[1:-1])<new]
                    for i in parent:
                        if i[0] >= ir or len(val1) == 1:
                            heappush(rule,min_heap)
                            break
                        t , p_val = ttest_ind(i[-1], val1, equal_var=True)
                        if not adjust:
                            sig = 0.05
                        else:
                            sig = 0.05/((2 ** len_tid) * (3 ** len_tid) * comb(num_col,len_tid))
                        if p_val > sig:
                            heappush(rule,min_heap)
                            break
                    if len(rule) == k-1:
                        heappush(rule,tuple([ir]+list(new)+[val1]))     
                else:
                    heappush(rule,min_heap)
                pro_a(k,me_1,rule,val,tid,num_col,new,sofar,measurement,pro,adjust)
                sofar.add(p)
            else:
                heappush(rule,min_heap)
        else:
            heappush(rule,min_heap)
            if upper_bound > min_heap[0] and not math.isnan(upper_bound):                
                heappushpop(rule, tuple([ir]+list(new)+[val1]))
                pro_a(k,me_1,rule,val,tid,num_col,new,sofar,measurement,pro,adjust)
                sofar.add(p)
    return rule